In [1]:
import pandas as pd
import numpy as numpy
from sklearn.neighbors import LocalOutlierFactor

In [23]:
og_data = pd.read_excel(rf'Files/TXS que el collector no ve NOV23 al final del mes.xlsx')

In [ ]:
df = og_data.copy()
df = df.set_index('CreatedAt')

X = pd.get_dummies(df[['Client', 'VendorCode', 
       'CollectMethod', 'TargetCountry',  'NetAmountUSD',
       'SourceCountry']])


from sklearn.ensemble import IsolationForest
IF = IsolationForest(random_state=0, contamination=0.1).fit(X)

df['Anomaly_scores']  = IF.decision_function(X)
df['is_anomaly']  = IF.predict(X)
df.sort_values(by='is_anomaly')
anomalias = df[df.is_anomaly==-1].sort_values(by='Anomaly_scores', ascending=True)

df.to_excel(rf'IF anomalies - NOV 23.xlsx', index=False)

# MSFT GUIDE

In [10]:
def get_zscore(value, mean, std):
    # calculate z-score or number of standard deviations from mean
    if (
        std == 0
        or std is None
        or str(std).lower() in ["nan", "none", "null"]
        or mean is None
    ):
        if value == 0.0:
            return 0.0
        elif value != 0:
            return np.log10(value + 1)
    ans = (value - mean) / std
    # only interested in increases
    ans = max(0.0, ans)
    # take log to dampen numbers
    ans = np.log10(ans + 1)
    return float(ans)



In [33]:
data.columns

Index(['CreatedAt', 'Id_x', 'TransactionId', 'Client', 'VendorCode', 'Status',
       'CollectMethod', 'TargetCountry', 'TargetCurrency', 'NetAmountUSD',
       'SendingAmount', 'RecipientAmount', 'SourceToUSDExchangeRate',
       'ExchangeRate_x', 'PplAmount', 'SourceCountry', 'SenderFirstName',
       'SenderLastName', 'SenderDocument', 'SenderPhoneNumber',
       'senderCountry', 'receiverFirstName', 'receiverLastName',
       'receiverDocument', 'receiverDocumentType', 'receiverPhoneNumber',
       'receiverCountry', 'receiverBankAccountNumber',
       'receiverBankAccountType', 'Date'],
      dtype='object')

In [38]:

data = og_data.copy()
data['Date'] = data.CreatedAt.dt.date
zscore_columns = [
    "NetAmountUSD"
]

means = [x + "_mean" for x in zscore_columns]
stds = [x + "_std" for x in zscore_columns]
zscores = [x + "_zscore" for x in zscore_columns]

ind = ['Client', 'VendorCode', 'Status', 'CollectMethod', 'TargetCountry', 'TargetCurrency','SourceCountry', 'SenderDocument', 'SenderPhoneNumber', 'receiverCountry', 'Date']

zscore = data[zscore_columns + ind]
zscore = zscore.fillna(0)

# getting means for user, domain and logon type combination
zscore[means] = zscore.groupby(['Client', 'VendorCode'])[zscore_columns].transform(
    "mean"
)

# getting standard deviation for user, domain and logon type combination
zscore[stds] = zscore.groupby(['Client', 'VendorCode'])[zscore_columns].transform(
    "std", ddof=1
)


In [40]:
zscore

,NetAmountUSD,Client,VendorCode,Status,CollectMethod,TargetCountry,TargetCurrency,SourceCountry,SenderDocument,SenderPhoneNumber,receiverCountry,Date,NetAmountUSD_mean,NetAmountUSD_std
0,5441.022042,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,CHL,00103233747,5.699874e+10,ESP,2023-11-22,814.473986,1067.893095
1,2721.651280,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,CHL,00277829045,5.699320e+10,ESP,2023-11-30,814.473986,1067.893095
2,2548.889692,SANTANDERCHL,COBRU,COMPLETED,BANK_ACCOUNT,COL,COP,CHL,00244993249,5.696789e+10,COL,2023-11-07,557.004832,977.200615
3,2230.651488,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,CHL,00052005191,5.695221e+10,ESP,2023-11-02,814.473986,1067.893095
4,2178.305170,SANTANDERCHL,TRANSFERZERO,COMPLETED,BANK_ACCOUNT,ESP,EUR,CHL,00109178497,5.696610e+10,ESP,2023-11-30,814.473986,1067.893095
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
576,1.770000,BANCO_ESTADO,ITALCAMBIO,COMPLETED,BANK_ACCOUNT,VEN,VEF,CHL,274083433,0.000000e+00,VEN,2023-11-17,50.473119,51.480984
577,1.430000,BANCO_ESTADO,ITALCAMBIO,COMPLETED,BANK_ACCOUNT,VEN,VEF,CHL,260520652,0.000000e+00,VEN,2023-11-18,50.473119,51.480984
578,1.392705,REMITEE,BANCOINDUSTRIAL,COMPLETED,BANK_ACCOUNT,ARG,ARS,ARG,41.879.837,5.491126e+12,ARG,2023-11-27,81.432082,111.086973
579,1.387071,REMITEE,BANCOINDUSTRIAL,COMPLETED,BANK_ACCOUNT,ARG,ARS,ARG,41.879.837,5.491126e+12,ARG,2023-11-30,81.432082,111.086973


In [41]:
zscore.groupby(['Client', 'VendorCode'])[zscore_columns].transform(
    "mean"
)

,NetAmountUSD
0,814.473986
1,814.473986
2,557.004832
3,814.473986
4,814.473986
...,...
576,50.473119
577,50.473119
578,81.432082
579,81.432082


In [ ]:

zscore = zscore.drop_duplicates(["DstDomain", "DstUser"])

zscore = zscore[means + stds + ["DstDomain", "DstUser"]]

data = data.merge(zscore, how="left", on=["DstDomain", "DstUser"])

# Calculating z scores
for column in zscore_columns:
    data[f"{column}_zscore"] = data.apply(
        lambda row: get_zscore(
            row[f"{column}"], row[f"{column}_mean"], row[f"{column}_std"]
        ),
        axis=1,
    )
# Display top 10 record
data.head()